In [1]:
import pandas
import xlrd

arrayData = []

rb = xlrd.open_workbook('echocardiogram.data.xls',formatting_info=True)
sheet = rb.sheet_by_index(0)
for rownum in range(sheet.nrows - 1):
    row = sheet.row_values(rownum + 1)
    values = []
    flag = True
    for i in range(len(row)):
         if i == 1 :
             if type(row[i]) is float : target = row[i]
             else: flag = False
         else:
             if type(row[i]) is float : value = row[i]
             else: flag = False
             values.append(value)
    if flag == True:
        
     values.append(target)   
     arrayData.append(values)


dataframe = pandas.DataFrame(arrayData)  

In [2]:
dataframe.head()

,0,1,2,3,4,5,6,7,8,9
0,11.0,71.0,0.0,0.260,9.000,4.600,14.0,1.00,1.000,0.0
1,19.0,72.0,0.0,0.380,6.000,4.100,14.0,1.70,0.588,0.0
2,16.0,55.0,0.0,0.260,4.000,3.420,14.0,1.00,1.000,0.0
3,57.0,60.0,0.0,0.253,12.062,4.603,16.0,1.45,0.788,0.0
4,19.0,57.0,0.0,0.160,22.000,5.750,18.0,2.25,0.571,1.0


In [3]:
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:9].astype(float)
y = dataset[:,9]

In [4]:
from sklearn import preprocessing
# normalize the data attributes
normalized_X = preprocessing.normalize(X)
# standardize the data attributes
standardized_X = preprocessing.scale(X)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(normalized_X , y, test_size=0.20)

In [23]:
from sklearn.neural_network import MLPClassifier

In [24]:
mlp = MLPClassifier(hidden_layer_sizes=(9, 1), 
                    max_iter=100,
                    alpha=1e-2,
                    solver='lbfgs', 
                    activation='logistic', 
                    learning_rate_init=.5)
mlp.fit(X_train, y_train)

MLPClassifier(activation='logistic', alpha=0.01, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(9, 1), learning_rate='constant',
       learning_rate_init=0.5, max_iter=100, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [28]:
print("Training set score: %f" % mlp.score(X_train, y_train))
print("Test set score: %f" % mlp.score(X_test, y_test))


Training set score: 0.703704
Test set score: 0.904762


In [29]:
import numpy
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
seed = 7
numpy.random.seed(seed)
estimators = []
estimators.append(('mlp', mlp))
pipeline = Pipeline(estimators)
kfold = KFold( shuffle=True, random_state=seed)
results = cross_val_score(pipeline, normalized_X, y, cv=kfold, pre_dispatch='5')
print("Result: %.2f%% +-(%.2f%%)" % (results.mean()*100, results.std()*100))

Result: 65.69% +-(7.34%)
